In [15]:
#function to pre process text data

# dataframe: dataframe object

# target_column_name: column on which stop words removal is needed

# new_column_name: new column to store the cleaned data

# 1. remove stop words

# 2. remove puncutations etc

# 3. remove special characters


import pandas as pd

import string
import re

import matplotlib.pyplot as plt
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
import numpy as np

import nltk

from nltk.corpus import stopwords
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

nltk.download('stopwords')

import nltk


from nltk.stem import PorterStemmer

ps = PorterStemmer()

# import tensorflow as tf

df = pd.read_csv("/Users/lifeifan/Desktop/ece1786/project/data1.csv")

df['cmp_code'] = df['cmp_code'].replace({'conservatism': 0, 'progressivism': 1})
df = df.rename(columns={'cmp_code': 'label'})

def remove_stop_words (dataframe,target_column_name,new_column_name) :
    dataframe[new_column_name] = dataframe[target_column_name].apply(lambda x:' '.join([item for item in x.split() if item not in stopwords.words('english')]))
    return dataframe


def remove_punctuations(dataframe,target_column_name,new_column_name):
    dataframe[new_column_name] = dataframe[target_column_name].apply(lambda x: "".join([char for char in x if char not in string.punctuation]))
    return dataframe

def stem_text(dataframe,target_column_name,new_column_name):
    dataframe[new_column_name] = dataframe[target_column_name].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))
    return dataframe




showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lifeifan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
cleaned_data = remove_stop_words(df,"text","cleaned_text")
print(cleaned_data)



                                                   text  label  \
0     But too many Americans have been left out and ...      1   
1               and the racial wealth gap remains wide,      1   
2                empowerment is better than resentment,      1   
3                    and bridges are better than walls.      1   
4     —an economy that grows incomes for working peo...      1   
...                                                 ...    ...   
1995  To prevent sectors becoming partially or wholl...      0   
1996     We will promote integration and British values      0   
1997  Being able to speak English is a fundamental p...      0   
1998  We have introduced tough new language tests fo...      0   
1999  Next, we will legislate to ensure that every p...      0   

                                           cleaned_text  
0                  But many Americans left left behind.  
1                       racial wealth gap remains wide,  
2                        empowerm

In [17]:
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [19]:
texts = cleaned_data.cleaned_text.values
labels = df.label.values

In [20]:
train_input, val_input, train_label, val_label=train_test_split (texts, labels)

In [21]:
print(len(val_input))

500


In [22]:
max_len = 0
for content in texts:
    max_len = max(max_len, len(content))
print('Max sentence length: ', max_len)


Max sentence length:  375


In [23]:
print("token",tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))

token [2021, 2116, 4841, 2187, 2187, 2369, 1012]


In [25]:
import torch
import tensorflow as tf
def mask_inputs_for_bert (text,max_len):
# Tokenize all of the sentences and map the tokens to thier word IDs. input_ids = []
    attention_masks = []
    input_ids = []
    i = 0
    # For every sentence...
    for content in text:

        encoded_dict = tokenizer.encode_plus(
            content,
            add_special_tokens = True, # Add '[CLS]' and '[SEP]' 
            max_length= max_len,
            # Sentence to encode.
            # Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True, # Construct attn. masks.
        )

        # Add the encoded sentence to the list.
        input_ids.append(encoded_dict['input_ids'])
        # And its attention mask (simply differentiates padding from non-padding). 
        attention_masks.append(encoded_dict['attention_mask'])

        #convert to tensor and return
    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)
    return input_ids, attention_masks

In [26]:
train_inp, train_mask = mask_inputs_for_bert (train_input,max_len) 
val_inp, val_mask = mask_inputs_for_bert (val_input, max_len) 
train_label =  tf.convert_to_tensor(train_label)
val_label =  tf.convert_to_tensor(val_label)
print("Train_input_shape", train_inp.shape) 
print("Train_mask_shape", train_mask.shape) 
print("Validation_input_shape", val_inp.shape) 
print("Validation_mask_shape", val_mask.shape) 
print("Train_labelshape", train_label.shape) 
print("Validation_label_shape", val_label.shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Train_input_shape (1500, 375)
Train_mask_shape (1500, 375)
Validation_input_shape (500, 375)
Validation_mask_shape (500, 375)
Train_labelshape (1500,)
Validation_label_shape (500,)


In [12]:
print(train_inp)

tf.Tensor(
[[  101  2057  2442 ...     0     0     0]
 [  101  4895 11787 ...     0     0     0]
 [  101  2057  2036 ...     0     0     0]
 ...
 [  101  3914  3952 ...     0     0     0]
 [  101  8037  2903 ...     0     0     0]
 [  101  7659  2610 ...     0     0     0]], shape=(1500, 375), dtype=int32)


In [2]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:

log_dir='/Users/lifeifan/Desktop/ece1786/project/'
model_save_path='/Users/lifeifan/Desktop/ece1786/project/bert.h5'
callbacks = [tf.keras.callbacks. ModelCheckpoint (filepath=model_save_path,
                                                    save_weights_only=True,
                                                    monitor= 'val_loss', 
                                                    mode='min',
                                                    save_best_only=True),
                                                    tf.keras.callbacks.TensorBoard (log_dir=log_dir)]
print('\nBert Model', model.summary())
loss = tf.keras.losses. SparseCategoricalCrossentropy (from_logits=True) 
metric = tf.keras.metrics. SparseCategoricalAccuracy('accuracy')
optimizer= tf.keras.optimizers.legacy.Adam (learning_rate=2e-5, epsilon=1e-08)
model.compile(loss=loss, optimizer=optimizer, metrics=[metric])

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

Bert Model None


In [15]:
history=model.fit([train_inp, train_mask],
                    train_label,
                    batch_size=32,
                    epochs=8,
                    validation_data=([val_inp, val_mask], val_label),
                    callbacks=callbacks)

94/94 [==============================] - 2097s 22s/step - loss: 0.6869 - accuracy: 0.5387 - val_loss: 0.6042 - val_accuracy: 0.6820


In [7]:
import matplotlib.pyplot as plt
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

NameError: name 'history' is not defined

In [3]:
import tensorflow as tf
from transformers import BertTokenizerFast,TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
model.load_weights('/Users/lifeifan/Desktop/ece1786/project/bert.h5')



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from sklearn.metrics import f1_score
preds = model.predict([val_inp,val_mask],batch_size=32)

NameError: name 'val_inp' is not defined

In [43]:
pred_labels = preds.logits.argmax(axis=1)
f1 = f1_score(val_label,pred_labels)
print('F1 score',f1)
print('Classification Report')
# print(classification_report(val_label,pred_labels,target_names=target_names))

print('Training and saving built model.....') 


F1 score 0.6940639269406392
Classification Report
Training and saving built model.....


In [6]:


import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Preprocess your single sentence
input_text = "The working class can surely win."
input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors="tf")
predictions = model(input_ids)
import numpy as np

logits = predictions.logits
probabilities = tf.nn.softmax(logits, axis=1)
print(probabilities)
predicted_class = tf.argmax(probabilities, axis=1)
print(predicted_class)

class_labels = ['con', 'pro']
predicted_label = class_labels[predicted_class.numpy()[0]]
print(predicted_class)

tf.Tensor([[0.21499395 0.785006  ]], shape=(1, 2), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64)
tf.Tensor([1], shape=(1,), dtype=int64)
